In [15]:
from math import log
import operator

def createDataSet1():
    
    dataSet = [('青年', '否', '否', '一般', '不同意'),
               ('青年', '否', '否', '好', '不同意'),
               ('青年', '是', '否', '好', '同意'),
               ('青年', '是', '是', '一般', '同意'),
               ('青年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '好', '不同意'),
               ('中年', '是', '是', '好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '好', '同意'),
               ('老年', '是', '否', '好', '同意'),
               ('老年', '是', '否', '非常好', '同意'),
               ('老年', '否', '否', '一般', '不同意')]
    # 特征集
    labels = ['年龄', '有工作', '有房子', '信贷情况']
    return dataSet,labels


def calcProbabilityEnt(dataSet):
    #样本点属于第一类的概率为p，即计算2p(1-p)中的p
    
    numEntries=len(dataSet)#数据量
    feaCounts=0
    fea1=dataSet[0][len(dataSet[0])-1]
    for featVec in dataSet:#每行数据
        if featVec[-1]==fea1:
            feaCounts+=1
    probabilityEnt=float(feaCounts)/numEntries#N(A,X=1)/N(A)；A：属性
    return probabilityEnt


def splitDataSet(dataSet,index,value):
    """
    划分数据集，提取额含有某个特征的某个属性的所有数据
    @param dataSet:数据集
    @param index:属性值所在的特征列
    @param value: 某个属性值
    @return retDataSet:含有某个特征的某个属性的数据集
    
    """
    retDataSet=[]#去除最优特征值属性后的集合
    for featVec in dataSet:
        if featVec[index]==value:
            reduceFeatVec=featVec[:index]+featVec[index+1:]#去掉index位置的属性
            retDataSet.append(reduceFeatVec)
    return retDataSet


#选择最优特征所在的列
def chooseBestFeatureToSplit(dataSet):
    numFeatures=len(dataSet[0])-1#特征总数
    if numFeatures==1:#是有一个特征
        return 0
    bestGini=1#最佳基尼系数
    bestFeature=-1#最优特征
    for i in range(numFeatures):
        uniqueVals=set(example[i] for example in dataSet)
        feaGini=0#定义特征值的基尼系数
        #计算每个特征的熵
        for value in uniqueVals:
            subDataSet=splitDataSet(dataSet,i,value)#对该特征进行分类，如：年龄：青年和非青年
            prob=len(subDataSet)/float(len(dataSet))
            probabilityEnt=calcProbabilityEnt(subDataSet)
            feaGini+=prob*(2*probabilityEnt*(1-probabilityEnt))#计算2*p*(1-p)
        if feaGini<bestGini:#选择最小基尼系数即为最优
            bestGini=feaGini
            bestFeature=i#最优特征的位置
    return bestFeature

#对最后一个特征分类，出现最多的类即为该属性类别
def majorityCnt(classList):
    classCount={}
    #计算每个类别出现的次数
    for vote in classList:
        try:
            classCount[vote]+=1
        except KeyError:
            classCount[vote]=1
    #出现最多的类别放在前面；operator.itemgetter(1):对第一个域来进行排序（即对出现次数排序）
    sortedClassCount=sorted(classCount.items(),key=operator.itemgetter(1),reverse=True)

    return sortedClassCount[0][0]#返回出现最多次的属性类别



def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]
    #只有一个类别
    if classList.count(classList[0])==len(classList):
        return classList[0]
    #当前数据只剩一列
    if len(dataSet[0])==1:
        return majorityCnt(classList)
    
    bestFeat=chooseBestFeatureToSplit(dataSet)#选择最优特征（所在的列）
    bestFeatLabel=labels[bestFeat]#最优特征
    del(labels[bestFeat])#删除最优特征
    uniqueVals=set(example[bestFeat] for example in dataSet)#去重
    myTree={bestFeatLabel:{}}
    for value in uniqueVals:
        subLabels=labels[:]#深拷贝
        #递归创建决策树
        myTree[bestFeatLabel][value]=createTree(splitDataSet(dataSet,bestFeat,value),subLabels)
    return myTree

if __name__=='__main__':
    dataSet,labels=createDataSet1()
    print(createTree(dataSet,labels))
    

    

    


{'有房子': {'否': {'有工作': {'否': '不同意', '是': '同意'}}, '是': '同意'}}


In [24]:
from math import log
import operator


def createDataSet1():
    """
    创造示例数据/读取数据
    @param dataSet: 数据集
    @return dataSet labels：数据集 特征集
    """
    # 数据集
    dataSet = [('青年', '否', '否', '一般', '不同意'),
               ('青年', '否', '否', '好', '不同意'),
               ('青年', '是', '否', '好', '同意'),
               ('青年', '是', '是', '一般', '同意'),
               ('青年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '一般', '不同意'),
               ('中年', '否', '否', '好', '不同意'),
               ('中年', '是', '是', '好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('中年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '非常好', '同意'),
               ('老年', '否', '是', '好', '同意'),
               ('老年', '是', '否', '好', '同意'),
               ('老年', '是', '否', '非常好', '同意'),
               ('老年', '否', '否', '一般', '不同意')]
    # 特征集
    labels = ['年龄', '有工作', '有房子', '信贷情况']
    return dataSet, labels


#根据特征A的a类属性进行分割数据
def splitDataSet(dataSet,index,val):
    n=len(dataSet)
    retDataSet=[]
    for i in range(n):
        if dataSet[i][index]==val:
            newData=dataSet[i][:index]+dataSet[i][index+1:]
            retDataSet.append(newData)
    return retDataSet

#计算2*p（1-p）中的p
def calP(dataSet):
    n=len(dataSet)
    count=0
    yes=dataSet[0][len(dataSet[0])-1]
    for i in range(n):
        if dataSet[i][len(dataSet[0])-1]==yes:
            count+=1
    p=count/n
    return p

#根据每个特征的每个属性的Gini系数，选择最优（小）的Gini对应的特征作为根节点；return 最优特征的列下标
def chooseBestFeature(dataSet):
    #二分类：Gini(D)=2*p*(1-p)
    #Gini(D,A)=D1/D*Gini(D1)+D2/D*Gini(D2)
    n=len(dataSet[0])-1
    if n==1:
        return 0
    bestGini=1
    bestFeatIndex=-1
    for i in range(n):
        uniqueFeature=set(example[i] for example in dataSet)
        featGini=0
        for val in uniqueFeature:
            subDataSet=splitDataSet(dataSet,i,val)
            prob=len(subDataSet)/float(len(dataSet))
            p=calP(subDataSet)
            featGini+=prob*(2*p*(1-p))
        if featGini<bestGini:
            bestGini=featGini
            bestFeatIndex=i
    return bestFeatIndex

#若只剩最后一列：该特征的属性选择出现次数最多的作为根节点
def lastFeature(classList):
    classCount={}
    for vote in FeatureList:
        try:
            classCount[vote]+=1
        except KeyError:
            classCount[vote]=1
    mostCount=sorted(classCount.items(),key=operater.itemgetter(1),reverse=True)
    return mostCount[0][0]

#递归创建决策树
def createTree(dataSet,labels):
    classList=[example[-1] for example in dataSet]
    #只有一个类别
    if classList.count(classList[0])==len(classList):
        return classList[0]
    #只剩1列
    if len(dataSet[0])==1:
        return lastFeature(classList)
    bestFeature=chooseBestFeature(dataSet)
    bestFeatureLabel=labels[bestFeature]
    del(labels[bestFeature])
    uniqueFeature=[example[bestFeature] for example in dataSet]
    myTree={bestFeatureLabel:{}}
    for val in uniqueFeature:
        subLabels=labels[:]
        myTree[bestFeatureLabel][val]=createTree(splitDataSet(dataSet,bestFeature,val),subLabels)
    return myTree


if __name__=="__main__":
    dataSet,labels=createDataSet1()
    print(createTree(dataSet,labels))



{'有房子': {'否': {'有工作': {'否': '不同意', '是': '同意'}}, '是': '同意'}}
